In [106]:
#필요 library import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Activation
from sklearn.preprocessing import minmax_scale
import autokeras as ak

In [107]:
#load data files
file = '1996-2020_U3.csv'
u1a = '2002-2004_U1a.csv'
u1b = '2002-2004_U1b.csv'
u2a = '2002-2004_U2a.csv'
u2b = '2002-2004_U2b.csv'
u3a = '2002-2004_U3a.csv'
u3b = '2002-2004_U3b.csv'
u4a = '2002-2004_U4a.csv'
u4b = '2002-2004_U4b.csv'
u5a = '2002-2004_U5a.csv'
obs_file = 'SST_merge.csv'

df = pd.read_csv(file, encoding='cp949')
df_u1a = pd.read_csv(u1a, encoding='cp949')
df_u1b = pd.read_csv(u1b, encoding='cp949')
df_u2a = pd.read_csv(u2a, encoding='cp949')
df_u2b = pd.read_csv(u2b, encoding='cp949')
df_u3a = pd.read_csv(u3a, encoding='cp949')
df_u3b = pd.read_csv(u3b, encoding='cp949') #결측값이 많아 test용으로 사용
df_u4a = pd.read_csv(u4a, encoding='cp949')
df_u4b = pd.read_csv(u4b, encoding='cp949')
df_u5a = pd.read_csv(u5a, encoding='cp949')
obs = pd.read_csv(obs_file, encoding='cp949') #해양수산부 바다누리 해양정보서비스에서 제공하는 수온, 기압 데이터


file_list = [df_u1a, df_u1b, df_u2a, df_u2b, df_u3a, df_u3b, df_u4a, df_u4b, df_u5a]
str_list = ['df_u1a', 'df_u1b', 'df_u2a', 'df_u2b', 'df_u3a', 'df_u3b', 'df_u4a', 'df_u4b', 'df_u5a']

#날짜 형식으로 변환 후 지점별 번호 부여
for i, j in zip(file_list, str_list):
    date = i['Year'].astype(str) + i['Month'].astype(str) + i['Day'].astype(str)
    i.insert(0, 'Date', date)
    i['Date'] = pd.to_datetime(i['Date'],format='%Y%m%d', errors='raise')

    if j == 'df_u1a' or j == 'df_u1b':
        i.insert(5, 'Station', 0)
    elif j == 'df_u2a' or j == 'df_u2b':
        i.insert(5, 'Station', 1)
    elif j == 'df_u3a' or 'df_u3b':
        i.insert(5, 'Station', 2)
    elif j =='df_u4a' or j == 'df_u4b':
        i.insert(5, 'Station', 3)
    else:
        i.insert(5, 'Station', 4)


drop_col = ['Year', 'Month', 'Day', 'Minute', 'Second']
for j in file_list:
    j.drop(drop_col, axis=1, inplace=True)

for k, l in zip(file_list, str_list):
    k.rename(columns = {'Ur Current speed (cm/s)' : l + '_current_speed_(cm/s)', 'Depth (meter)' : l + '_Depth_(m)'}, inplace=True)

obs['Date'] = obs['Date'].astype('str')

def merge_files(file1, file2):
    file1['Date'] = file1['Date'].astype('str')
    file1 = pd.merge(file1, file2, on=['Date', 'Hour'])
    file1 = np.array(file1)

    return file1

#코드 예쁘게? ##########################@@@@@@@@@@@@@@@@ 루프로 하면 안 됨 덮어씌우기 때문인듯?
df_u1a = merge_files(df_u1a, obs)
df_u1b = merge_files(df_u1b, obs)
df_u2a = merge_files(df_u2a, obs)
df_u2b = merge_files(df_u2b, obs)
df_u3a = merge_files(df_u3a, obs)
df_u3b = merge_files(df_u3b, obs)
df_u4a = merge_files(df_u4a, obs)
df_u4b = merge_files(df_u4b, obs)
df_u5a = merge_files(df_u5a, obs)

df_u3b

array([['2002-11-29', 18, 2, ..., -2.4852e-07, 14.1, 1019.0],
       ['2002-11-29', 19, 2, ..., -2.35876e-06, 14.1, 1018.51],
       ['2002-11-29', 20, 2, ..., -1.10679e-05, 14.1, 1018.02],
       ...,
       ['2003-06-05', 4, 2, ..., -2.523660074, 17.44, 1006.7142857142856],
       ['2003-06-05', 5, 2, ..., -2.501656526, 17.4, 1006.8571428571428],
       ['2003-06-05', 6, 2, ..., -2.48076972, 17.34, 1007.0]],
      dtype=object)

In [108]:
#encoding 두 번 하는 방식 말고 바꿀 수 있으면 바꾸기 아니면 말고
file_list = [df_u1a, df_u1b, df_u2a, df_u2b, df_u3a, df_u3b, df_u4a, df_u4b, df_u5a]
df_all = np.concatenate(file_list)

#labelencoding
le = LabelEncoder()
df_all[:, 0] = le.fit_transform(df_all[:, 0])

#minmax scaling
df_all[:, 4] = minmax_scale(df_all[:,4])
df_all[:, -2] = minmax_scale(df_all[:, -2])
df_all[:, -1] = minmax_scale(df_all[:, -1])

df_u3b = df_all[(df_all[:, 2] == 2) & (df_all[:, 3] == 2240)]
df_u3b = df_u3b.astype(float)

df_u3b

array([[  1.        ,  18.        ,   2.        , ...,   0.30672544,
          0.37567406,   0.70268034],
       [  1.        ,  19.        ,   2.        , ...,   0.30672539,
          0.37567406,   0.69457313],
       [  1.        ,  20.        ,   2.        , ...,   0.30672517,
          0.37567406,   0.68646592],
       ...,
       [168.        ,   4.        ,   2.        , ...,   0.24279786,
          0.57579389,   0.4994091 ],
       [168.        ,   5.        ,   2.        , ...,   0.24335524,
          0.57339724,   0.50177271],
       [168.        ,   6.        ,   2.        , ...,   0.24388433,
          0.56980228,   0.50413633]])

In [109]:
file_list = [df_u1a, df_u1b, df_u2a, df_u2b, df_u3a, df_u4a, df_u4b, df_u5a] #df_u3b 제외
df_all = np.concatenate(file_list)
le = LabelEncoder()
df_all[:, 0] = le.fit_transform(df_all[:, 0])
df_all[:, 4] = minmax_scale(df_all[:, 4])
df_all[:, -2] = minmax_scale(df_all[:, -2])
df_all[:, -1] = minmax_scale(df_all[:, -1])

df_all = df_all.astype(float)
df_all

array([[2.00000000e+00, 1.10000000e+01, 0.00000000e+00, ...,
        3.06725486e-01, 3.69682445e-01, 7.03011251e-01],
       [2.00000000e+00, 1.20000000e+01, 0.00000000e+00, ...,
        3.06725820e-01, 3.69682445e-01, 6.92753144e-01],
       [2.00000000e+00, 1.30000000e+01, 0.00000000e+00, ...,
        3.06727199e-01, 3.69682445e-01, 6.86135010e-01],
       ...,
       [4.33000000e+02, 6.00000000e+00, 2.00000000e+00, ...,
        6.41518331e-01, 1.77950869e-01, 6.19953673e-01],
       [4.33000000e+02, 7.00000000e+00, 2.00000000e+00, ...,
        6.42127778e-01, 1.77950869e-01, 6.19953673e-01],
       [4.33000000e+02, 8.00000000e+00, 2.00000000e+00, ...,
        6.42956242e-01, 1.77950869e-01, 6.17140966e-01]])

In [112]:
#train, test set
x_tr = pd.DataFrame(df_all).iloc[:, [0,1,2,3,5,6]]
y_tr = pd.DataFrame(df_all[:, 4]) # 유속

x_test = pd.DataFrame(df_u3b).iloc[:, [0,1,2,3,5,6]]
y_test = pd.DataFrame(df_u3b[:, 4]) #정답값
x_tr

,0,1,2,3,5,6
0,2.0,11.0,0.0,1760.0,0.369682,0.703011
1,2.0,12.0,0.0,1760.0,0.369682,0.692753
2,2.0,13.0,0.0,1760.0,0.369682,0.686135
3,2.0,14.0,0.0,1760.0,0.369682,0.686135
4,2.0,15.0,0.0,1760.0,0.369682,0.686135
...,...,...,...,...,...,...
84551,433.0,4.0,2.0,2060.0,0.178550,0.607048
84552,433.0,5.0,2.0,2060.0,0.177951,0.619954
84553,433.0,6.0,2.0,2060.0,0.177951,0.619954
84554,433.0,7.0,2.0,2060.0,0.177951,0.619954


In [111]:
#AutoKeras
model_2 = ak.StructuredDataRegressor(max_trials=10, seed=42, overwrite=True)
model_2.fit(x_tr, y_tr, verbose=1, epochs=50)
pred2 = model_2.predict(x_test)

ValueError: Expect the data to StructuredDataInput to have shape (batch_size, num_features), but got input shape [32, 6, 1].

In [ ]:
#AutoKeras 결과 확인
plt.plot(pred2, color='yellow')
# plt.xlim(75000, 175000)
plt.plot(y_test, alpha=0.5)
plt.show()

plt.plot(y_test)
plt.show()

plt.plot(pred2)
plt.ylim([0, 1])
plt.show()

pred2

In [ ]:
#LSTM 학습을 위해 차원 추가
x_tr = np.expand_dims(x_tr, -1)
y_tr = np.expand_dims(y_tr, -1)
x_test = np.expand_dims(x_test, -1)

In [ ]:
#LSTM
model = Sequential()
model.add(CuDNNLSTM(50, return_sequences=True))
model.add(CuDNNLSTM(1, return_sequences=False))
model.add(Activation('relu'))
model.compile(optimizer='adam', loss='MSE', metrics=['accuracy'])
model.fit(x_tr, y_tr, epochs=100, batch_size=128, shuffle=False)
pred = model.predict(x_test)

In [ ]:
#LSTM 결과 확인
plt.plot(pred, color='yellow')
# plt.xlim(75000, 175000)
plt.plot(y_test, alpha=0.5)
plt.show()

plt.plot(y_test)
plt.show()

plt.plot(pred)
plt.ylim([0, 0.5])
plt.show()

pred